In [24]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(dbname="adsdb", user="postgres", password="postgresadmin")

In [125]:
df_morticd = pd.read_sql('SELECT * from formatted.morticd_7_22b4_1642871513;', conn)
df_countries = pd.read_sql('SELECT * from formatted.countrycodes_0_8c41_1642871517;', conn)

In [126]:
# Check if morticd countries are in countrycode country
df_morticd['CountryWrongCode'] = df_morticd['Country'].apply(lambda x: x in df_countries.Country)

In [164]:
# Check Size
df_morticd['CountryWrongSize'] = df_morticd['Country'].apply(lambda x: x != None and len(str(x))  != 4)
df_morticd['Admin1WrongSize'] = df_morticd['Admin1'].apply(lambda x: x != None and len(str(x)) != 3)
df_morticd['SubDivWrongSize'] = df_morticd['SubDiv'].apply(lambda x: x != None and len(str(x)) != 3)
df_morticd['YearWrongSize'] = df_morticd['Year'].apply(lambda x: x != None and len(str(x)) != 4)
df_morticd['ListWrongSize'] = df_morticd['List'].apply(lambda x: x != None and len(str(x)) != 3)
df_morticd['CauseWrongSize'] = df_morticd['Cause'].apply(lambda x: x != None and len(str(x)) > 4)
df_morticd['SexWrongSize'] = df_morticd['Sex'].apply(lambda x: x != None and len(str(x)) != 1)
df_morticd['FrmatWrongSize'] = df_morticd['Frmat'].apply(lambda x: x != None and len(str(x)) != 2)
df_morticd['IM_FrmatWrongSize'] = df_morticd['IM_Frmat'].apply(lambda x: x != None and len(str(x)) != 2)

In [165]:
names = df_morticd.filter(regex=("Deaths*")).columns
for death in names:    
    df_morticd[death + 'WrongSize'] = df_morticd[death].apply(lambda x: len(str(x)) > 8)
   

In [152]:
df_morticd

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths21WrongSizeWrongSizeWrongSize,Deaths22WrongSizeWrongSizeWrongSize,Deaths23WrongSizeWrongSizeWrongSize,Deaths24WrongSizeWrongSizeWrongSize,Deaths25WrongSizeWrongSizeWrongSize,Deaths26WrongSizeWrongSizeWrongSize,IM_Deaths1WrongSizeWrongSizeWrongSize,IM_Deaths2WrongSizeWrongSizeWrongSize,IM_Deaths3WrongSizeWrongSizeWrongSize,IM_Deaths4WrongSizeWrongSizeWrongSize
0,1125,None,None,1955,07A,A000,1,01,08,107822,...,False,False,False,False,False,False,False,False,False,False
1,1125,None,None,1955,07A,A000,2,01,08,105750,...,False,False,False,False,False,False,False,False,False,False
2,1125,None,None,1955,07A,A001,1,01,08,1302,...,False,False,False,False,False,False,False,False,False,False
3,1125,None,None,1955,07A,A001,2,01,08,739,...,False,False,False,False,False,False,False,False,False,False
4,1125,None,None,1955,07A,A002,1,01,08,69,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281744,4340,None,None,1960,07B,202,2,02,08,4,...,False,False,False,False,False,False,False,False,False,False
281745,4340,None,None,1960,07B,203,1,02,08,13,...,False,False,False,False,False,False,False,False,False,False
281746,4340,None,None,1960,07B,203,2,02,08,13,...,False,False,False,False,False,False,False,False,False,False
281747,4340,None,None,1960,07B,205,1,02,08,0,...,False,False,False,False,False,False,False,False,False,False


In [170]:
wrongs = df_morticd.filter(regex=(".Wrong*")).columns
wrongs

Index(['CountryWrongCode', 'CountryWrongSize', 'Admin1WrongSize',
       'SubDivWrongSize', 'YearWrongSize', 'ListWrongSize', 'CauseWrongSize',
       'SexWrongSize', 'FrmatWrongSize', 'IM_FrmatWrongSize',
       ...
       'Deaths21WrongSizeWrongSizeWrongSizeWrongSize',
       'Deaths22WrongSizeWrongSizeWrongSizeWrongSize',
       'Deaths23WrongSizeWrongSizeWrongSizeWrongSize',
       'Deaths24WrongSizeWrongSizeWrongSizeWrongSize',
       'Deaths25WrongSizeWrongSizeWrongSizeWrongSize',
       'Deaths26WrongSizeWrongSizeWrongSizeWrongSize',
       'IM_Deaths1WrongSizeWrongSizeWrongSizeWrongSize',
       'IM_Deaths2WrongSizeWrongSizeWrongSizeWrongSize',
       'IM_Deaths3WrongSizeWrongSizeWrongSizeWrongSize',
       'IM_Deaths4WrongSizeWrongSizeWrongSizeWrongSize'],
      dtype='object', length=130)

In [174]:
for wrong_col in wrongs:
   if len(df_morticd[df_morticd[wrong_col] == True]) > 0:
      print(wrong_col)